In [14]:
# Use wfdb to access the PhysioNet Data
import wfdb
import plotly.graph_objs as go
import plotly.offline as pyo

In [11]:
# Load both the header (.hea) and the signal data (.mat) using wfdb
record = wfdb.rdrecord('0284_001_004_ECG')

In [16]:
# Print the contents of the header file
print("Record Name:", record.record_name)
print("Sampling Frequency:", record.fs)
print("Signal Length (samples):", record.sig_len)
print("Number of Signals (channels):", record.n_sig)
print("Signal Names:", record.sig_name)
print("Signal Units:", record.units)

# Access the signal data (numpy array)
signal_data = record.p_signal.flatten()  # Assuming one channel and flatten to 1D

# Check the shape of the signal data
print("Signal Data Shape:", signal_data.shape)


Record Name: 0284_001_004_ECG
Sampling Frequency: 500
Signal Length (samples): 1578500
Number of Signals (channels): 1
Signal Names: ['ECG']
Signal Units: ['nu']
Signal Data Shape: (1578500,)


In [17]:
# Create the Plotly plot
trace = go.Scatter(
    y=signal_data,
    mode='lines',
    name='ECG Signal'
)

layout = go.Layout(
    title='Scrollable ECG Signal',
    xaxis=dict(
        title='Time (samples)',
        rangeslider=dict(visible=True),  # Add a range slider to make it scrollable
    ),
    yaxis=dict(
        title='Amplitude',
    ),
    width=1200,  # Set a wider plot
    height=500
)

fig = go.Figure(data=[trace], layout=layout)

# Show the plot in an interactive window
pyo.plot(fig, filename='scrollable_ecg_plot.html')

'scrollable_ecg_plot.html'